# PoC: WeCookio multiagent CrewAI INC

This notebook demonstrates the WeCookio CrewAI implementation with cost-optimized model selection using uv package manager.

### Process.Sequential without delegation

### Installing Required Packages with uv

In [5]:
# Install required packages using uv
!uv add ipykernel crewai crewai-tools boto3 botocore pyyaml ipywidgets

Resolved 224 packages in 0.75ms
Audited 213 packages in 0.07ms


### Import Libraries and Load Configuration

In [6]:
import yaml
from crewai import Crew, Agent, Task, Process
import boto3
import json

print("Libraries imported successfully!")

Libraries imported successfully!


### Loading AWS Enviroment variables

In [7]:
from dotenv import load_dotenv
import os

load_dotenv()
AWS_ACCESS_KEY_ID=os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY=os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_DEFAULT_REGION=os.getenv("AWS_REGION")

### Load Configuration Files

In [8]:
#Check base folder for relative paths
import os
print (os.getcwd())
prefix = "crew/latest/"
# Load the integration configuration
with open(prefix+'config/agent-sequential/integration_config.yaml', 'r') as file:
    integration_config = yaml.safe_load(file)
# Load the crew configuration
with open(prefix+'config/agent-sequential/crew_config.yaml', 'r') as file:
    crew_config = yaml.safe_load(file)
with open(prefix+'config/agent-sequential/agents.yaml', 'r') as file:
    agents_config = yaml.safe_load(file)
with open(prefix+'config/agent-sequential/tasks.yaml', 'r') as file:
    tasks_config = yaml.safe_load(file)

print("Configuration files loaded successfully!")
print(f"Integration config keys: {list(integration_config.keys())}")
print(f"Crew Orchestration keys: {list(crew_config.keys())}")
print(f"Agents config keys: {list(agents_config.keys())}")
print(f"Tasks config keys: {list(tasks_config.keys())}")

/home/e2its/dev/weCookio/researches
Configuration files loaded successfully!
Integration config keys: ['aws_bedrock', 'database', 'api_endpoints']
Crew Orchestration keys: ['imports', 'crew', 'agents', 'tasks', 'workflow', 'model_assignment', 'execution', 'memory', 'input_schema', 'output_schema', 'cost_optimization', 'performance_metrics', 'quality_standards', 'monitoring', 'example_input', 'example_output']
Agents config keys: ['Culinary Coordinator', 'Dietary Compliance Specialist', 'Ingredient Substitution Expert', 'Culinary Experience Optimizer', 'Quality Assurance Chef']
Tasks config keys: ['Initial Recipe Analysis', 'Dietary Compliance Validation', 'Ingredient Substitution Strategy', 'Culinary Experience Enhancement', 'Final Quality Assurance']


###  Model Selection based on complexity

In [9]:
def select_model_for_complexity(complexity, integration_config):
    """Select the appropriate model based on task complexity"""
    try:
        complexity = complexity + "_model"
        return integration_config["aws_bedrock"][complexity]
    except KeyError:
        raise ValueError(f"Invalid complexity level: {complexity}")

# Example usage
high_complexity_model = select_model_for_complexity("high_complexity", integration_config)
medium_complexity_model = select_model_for_complexity("medium_complexity", integration_config)
low_complexity_model = select_model_for_complexity("low_complexity", integration_config)
ultra_low_complexity_model = select_model_for_complexity("ultra_low_complexity", integration_config)

print(f"High complexity model: {high_complexity_model['model_id']}")
print(f"Medium complexity model: {medium_complexity_model['model_id']}")
print(f"Low complexity model: {low_complexity_model['model_id']}")
print(f"Ultra-Low complexity model: {ultra_low_complexity_model['model_id']}")

High complexity model: bedrock/amazon.nova-premier-v1:0
Medium complexity model: bedrock/amazon.nova-pro-v1:0
Low complexity model: bedrock/amazon.nova-lite-v1:0
Ultra-Low complexity model: bedrock/amazon.nova-micro-v1:0


### Create Agents with Model-Specific Configuration

In [10]:
from crewai import LLM
from pprint import pprint

def create_agent_with_model(agent_config, model_config):
    """Create an agent with specific model configuration"""
    llm = LLM(
        model=model_config["model_id"],
        region_name=AWS_DEFAULT_REGION,
        max_tokens=model_config.get("model_kwargs").get("max_tokens", 4000),
        temperature=model_config.get("model_kwargs").get("temperature", 0.6),
        top_p=model_config.get("model_kwargs").get("top_p", 0.9),
        api_key=AWS_SECRET_ACCESS_KEY
    )
    return Agent(
        role=agent_config["role"],
        goal=agent_config["goal"],
        backstory=agent_config["backstory"],
        verbose=agent_config.get("verbose", True),
        allow_delegation=agent_config.get("allow_delegation", False),
        prompt_template=agent_config.get("prompt", None),
        reasoning=agent_config.get("reasoning", False),
        max_reasoning_attempts=agent_config.get("max_reasoning_attempts", 3),
        #response_template="{{ .Response }}",
        max_iterations=model_config.get("model_kwargs").get("max_iterations", 3),
        max_retry_limit=model_config.get("model_kwargs").get("max_retry_limit", 2),
        max_execution_time=model_config.get("model_kwargs").get("max_execution_time", 30),
        llm=llm
    )

def print_agent_info(agent, model_config) -> None:
    """Print agent creation information."""
    print(
        f"Created agent: {agent.id}\n"
        f"Agent Role: {agent.role}\n"
        f"Using model: {model_config.get('model_id')}"
    )
    print("\n")

In [11]:

#agent_name = "Culinary Coordinator"
agent_config = agents_config["Culinary Coordinator"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(crew_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
culinary_coordinator_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(culinary_coordinator_agent, model_config)

#agent_name = "Dietary Compliance Specialist"
agent_config = agents_config["Dietary Compliance Specialist"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(crew_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
dietary_compliance_specialist_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(dietary_compliance_specialist_agent, model_config)

agent_name = "Ingredient Substitution Expert"
agent_config = agents_config["Ingredient Substitution Expert"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(crew_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
ingredient_substitution_expert_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(ingredient_substitution_expert_agent, model_config)

agent_name = "Culinary Experience Optimizer"
agent_config = agents_config["Culinary Experience Optimizer"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(crew_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
culinary_experience_optimizer_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(culinary_experience_optimizer_agent, model_config)

agent_name = "Quality Assurance Chef"
agent_config = agents_config["Quality Assurance Chef"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(crew_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
quality_assurance_chef_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(quality_assurance_chef_agent, model_config)





Created agent: 5ca32215-1f2d-4db7-acd4-ea3033eefa3a
Agent Role: Master Chef & Project Manager
Using model: bedrock/amazon.nova-lite-v1:0


Created agent: 3c7d257d-efa9-4e00-b7cb-0f5b9cdd4c34
Agent Role: Nutritionist & Food Safety Expert
Using model: bedrock/amazon.nova-micro-v1:0


Created agent: 3fe099a4-2b4b-4dba-8bbc-6aa087ec909a
Agent Role: Culinary Innovation Specialist
Using model: bedrock/amazon.nova-pro-v1:0


Created agent: aaf818fe-cbbe-4ba2-afec-ddee40ad5a00
Agent Role: Gastronomic Enhancement Specialist
Using model: bedrock/amazon.nova-lite-v1:0


Created agent: c4265662-92d5-4282-ac89-128278af72a5
Agent Role: Final Validation & Presentation Expert
Using model: bedrock/amazon.nova-lite-v1:0




### Create Tasks with Complexity-Based Model Selection

In [12]:
def create_task_with_agent(task_config, agent):
    """Create a task with the appropriate agent based on complexity"""
    return Task(
        description=task_config["description"],
        expected_output=task_config["expected_output"],
        agent=agent,
        async_execution=task_config.get("async_execution", False)
    )

# Example: Create a task for the coordinator
#task_name="Initial Recipe Analysis"
#inital_recipe_config = tasks_config[task_name]
#pprint.pprint(inital_recipe_config)
#print(f"Description type: {type(inital_recipe_config.get("description", "Not Found"))}")
#print(f"Expected output type: {type(inital_recipe_config.get("expected_output", "Not Found"))}")
#print(f"Template type: {type(inital_recipe_config.get("Template", "Not Found"))}")
#print(f"Async execution type: {type(inital_recipe_config.get("async_execution", "Not Found"))}")

def print_task_info(task) -> None:
    """Print information about a created task."""
    print(f"Created task: {task.id}")
    print(f"Task description: {task.description}")
    print(f"Assigned to agent: {task.agent.role}")
    print("\n")

In [13]:
# task_name = "Initial Recipe Analysis"
task_config = tasks_config["Initial Recipe Analysis"]
inital_recipe_analysis_task = create_task_with_agent(task_config, eval(task_config["agent"]))

print(f"Created task: {inital_recipe_analysis_task.id}")
print(f"Task description: {inital_recipe_analysis_task.description}")
print(f"Assigned to agent: {inital_recipe_analysis_task.agent.role}")
print("\n")

# task_name = "Dietary Compliance Validation"
task_config = tasks_config["Dietary Compliance Validation"]
dietary_compliance_validation_task = create_task_with_agent(task_config, eval(task_config["agent"]))

print(f"Created task: {dietary_compliance_validation_task.id}")
print(f"Task description: {dietary_compliance_validation_task.description}")
print(f"Assigned to agent: {dietary_compliance_validation_task.agent.role}")
print("\n")

task_name = "Ingredient Substitution Strategy"
task_config = tasks_config["Ingredient Substitution Strategy"]
ingredient_substitution_strategy_task = create_task_with_agent(task_config, eval(task_config["agent"]))

print(f"Created task: {ingredient_substitution_strategy_task.id}")
print(f"Task description: {ingredient_substitution_strategy_task.description}")
print(f"Assigned to agent: {ingredient_substitution_strategy_task.agent.role}")
print("\n")

# task_name = "Culinary Experience Enhancement"
task_config = tasks_config["Culinary Experience Enhancement"]
culinary_experience_enhancement_task = create_task_with_agent(task_config, eval(task_config["agent"]))

print(f"Created task: {culinary_experience_enhancement_task.id}")
print(f"Task description: {culinary_experience_enhancement_task.description}")
print(f"Assigned to agent: {culinary_experience_enhancement_task.agent.role}")
print("\n")

# task_name = "Final Quality Assurance"
task_config = tasks_config["Final Quality Assurance"]
final_quality_assurance_task = create_task_with_agent(task_config, eval(task_config["agent"]))

print(f"Created task: {final_quality_assurance_task.id}")
print(f"Task description: {final_quality_assurance_task.description}")
print(f"Assigned to agent: {final_quality_assurance_task.agent.role}")
print("\n")

Created task: f03827cd-d6ba-4d4a-a5bf-8c2894be6809
Task description: **OBJECTIVE**: Create a plan to analyze and replace diatary restrictions in the original recipe and user requirements to ensure 100% dietary compliance while maximizing culinary experience and maintaining authentic flavor/texture similarity.

**CRITICAL**: Zero tolerance for dietary restriction violations - this is your top priority.
**CRITICAL**: Maximize culinary experience and maintain flavor/texture similarity to original.

**Recipe Context**:
- Recipe: {{food_name}}
- Servings: {{servings}}
- Dietary Restrictions: {{intolerances}}, {{exclusions}}
- Preferences: {{preferences}}
- Country: {{country}}
- Output Language: {{output_language}}

**CRITICAL LANGUAGE RULE**: Provide ALL output in {{output_language}} language only. No mixed languages allowed.
    

Assigned to agent: Master Chef & Project Manager


Created task: 0c40fa04-4f5e-403d-a111-72719243cf58
Task description: **OBJECTIVE**: Validate all ingredients 

### Create Crew 

In [14]:
# Create a simple crew for demonstration
embedder={
        "provider": "bedrock",
        "config": {
            "model": "amazon.titan-embed-text-v2:0"
        }
    }

crew = Crew(
    agents=[culinary_coordinator_agent,  ingredient_substitution_expert_agent, culinary_experience_optimizer_agent, quality_assurance_chef_agent],
    tasks=[inital_recipe_analysis_task, ingredient_substitution_strategy_task, culinary_experience_enhancement_task, final_quality_assurance_task],
    verbose=True,
    memory=False, 
    #memory=True,
    #embedder=embedder,
    process=eval(crew_config.get("execution").get("process")),
    cache=crew_config.get("cache", True),
    #planning=True,
    #planner=select_model_for_complexity("low_complexity", integration_config)
)

print("Crew created successfully!")
print("Flow model: ", crew_config.get("execution").get("process"))
print(f"Number of agents: {len(crew.agents)}")
print(f"Number of tasks: {len(crew.tasks)}")
print(f"Memory enabled: {crew.memory}")
print(f"Cache enabled: {crew.cache}")

Crew created successfully!
Flow model:  Process.sequential
Number of agents: 4
Number of tasks: 4
Memory enabled: False
Cache enabled: True


### Recipe Parameters

In [15]:
# Example recipe for analysis
recipe_request = {
    "food_name": "Canelones de ternera XXL al Pedro Ximenez",
    "servings": 4,
    "intolerances": [
        "Milk",
        "Eggs",
        "Beef",
        "Corn",
        "Gluten",
        "Rice",
        "Garlic"
    ],
    "exclusions": ["Pepper"],
    "preferences": [],
    "output_language": "es",
    "country": "ES"
}

print("Sample request for analysis:")
print(json.dumps(recipe_request, indent=2))

Sample request for analysis:
{
  "food_name": "Canelones de ternera XXL al Pedro Ximenez",
  "servings": 4,
  "intolerances": [
    "Milk",
    "Eggs",
    "Beef",
    "Corn",
    "Gluten",
    "Rice",
    "Garlic"
  ],
  "exclusions": [
    "Pepper"
  ],
  "preferences": [],
  "output_language": "es",
  "country": "ES"
}


### Run the Crew (Requires AWS Credentials)

In [16]:
# Uncomment the following lines if you have AWS credentials configured
# and want to actually run the crew

try:
    result = crew.kickoff(inputs=recipe_request)
    print("\nCrew execution result:")
    print(result)
except Exception as e:
    print(f"Error running crew: {e}")
    print("Make sure you have AWS credentials configured and the required permissions.")

#print("Notebook execution completed successfully!")
#print("\nTo run the actual crew:")
#print("1. Configure AWS credentials")
#print("2. Ensure Bedrock access permissions")
#print("3. Uncomment the execution code above")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 41419c2f-a41c-4c51-8cde-53cebc9544fa                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Master Chef & Project Manager                                                                           │
│                                                                                                                 │
│  Task: **OBJECTIVE**: Create a plan to analyze and replace diatary restrictions in the original recipe and      │
│  user requirements to ensure 100% dietary compliance while maximizing culinary experience and maintaining       │
│  authentic flavor/texture similarity.                                                                           │
│                                                                                                                 │
│  **CRITICAL**: Zero tolerance for dietary restriction violations - this is your top priority.                   │
│  **CRITICAL**: Maximize culinary experience and maintain flavor/texture similarity to original.                 │
│                                                                                                                 │
│  **Recipe Context**:                                                                                            │
│  - Recipe: {Canelones de ternera XXL al Pedro Ximenez}                                                          │
│  - Servings: {4}                                                                                                │
│  - Dietary Restrictions: {['Milk', 'Eggs', 'Beef', 'Corn', 'Gluten', 'Rice', 'Garlic']}, {['Pepper']}           │
│  - Preferences: {[]}                                                                                            │
│  - Country: {ES}                                                                                                │
│  - Output Language: {es}                                                                                        │
│                                                                                                                 │
│  **CRITICAL LANGUAGE RULE**: Provide ALL output in {es} language only. No mixed languages allowed.              │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭────────────────────────────────────────────────── Tool Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  Tool Usage Failed                                                                                              │
│  Name: Delegate work to coworker                                                                                │
│  Error: Arguments validation failed: 2 validation errors for DelegateWorkToolSchema                             │
│  task                                                                                                           │
│    Input should be a valid string [type=string_type, input_value={'description': 'Investig...ic flavor and      │
│  texture.'}, input_type=dict]                                                                                   │
│      For further information visit https://errors.pydantic.dev/2.11/v/string_type                               │
│  context                                                                                                        │
│    Input should be a valid string [type=string_type, input_value={'description': "The reci...ten, rice, and     │
│  garlic."}, input_type=dict]                                                                                    │
│      For further information visit https://errors.pydantic.dev/2.11/v/string_type                               │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Final Validation & Presentation Expert                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Informe de Aseguramiento de Calidad**                                                                        │
│                                                                                                                 │
│  **Validación de Cumplimiento (Usando Experiencia Dietética):**                                                 │
│  - [ ] Todas las restricciones dietéticas estrictamente respetadas                                              │
│  - [ ] Sin alérgenos presentes                                                                                  │
│  - [ ] Preferencias acomodadas                                                                                  │
│  - [ ] No se han utilizado términos genéricos (sin gluten, sin lácteos, vegano, etc.)                           │
│                                                                                                                 │
│                                                                                                                 │
│  **Receta Final**: Canelones de "Ternera" XXL al Pedro Ximenez (4 porciones)                                    │
│                                                                                                                 │
│  **Ingredientes:**                                                                                              │
│  - 8 fideos de lasaña sin gluten                                                                                │
│  - 1 taza (240 ml) de leche de soja o de almendra sin azúcar                                                    │
│  - 1 taza (240 ml) de caldo de verduras                                                                         │
│  - 2 cucharadas (30 ml) de maicena                                                                              │
│  - 1 cucharada (15 ml) de harina de linaza + 3 cucharadas (45 ml) de agua (o sustituto de huevo comercial)      │
│  - 1 taza (240 g) de carne de res vegetal a base de soja o champiñones finamente texturizados                   │
│  - 1 cebolla pequeña, finamente picada                                                                          │
│  - 2 cucharadas (30 ml) de aceite de oliva                                                                      │
│  - 1 taza (140 g) de puerros finamente picados u cebollas verdes                                                │
│  - 1 taza (140 g) de coliflor picada o arroz de coliflor                                                        │
│  - 1 taza (240 ml) de jerez Pedro Ximenez                                                                       │
│  - 2 cucharadas (30 ml) de pasta de tomate                                                                      │
│  - 1 cucharadita (5 ml) de pimentón ahumado                                                                     │
│  - 1 cucharadita (5 ml) de comino                                                                               │
│  - Sal y pimienta al gusto                                                                                      │
│  - Aceite de oliva con infusión de ajo o polvo de cebolla (para sustituto de ajo)                               │
│                                                                                                                 │
│  **Instrucciones:**                                    

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b2570cf8-be2a-442d-bf8e-557a2feb59bb                                                                     │
│  Agent: Final Validation & Presentation Expert                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 41419c2f-a41c-4c51-8cde-53cebc9544fa                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **Informe de Aseguramiento de Calidad**                                                          │
│                                                                                                                 │
│  **Validación de Cumplimiento (Usando Experiencia Dietética):**                                                 │
│  - [ ] Todas las restricciones dietéticas estrictamente respetadas                                              │
│  - [ ] Sin alérgenos presentes                                                                                  │
│  - [ ] Preferencias acomodadas                                                                                  │
│  - [ ] No se han utilizado términos genéricos (sin gluten, sin lácteos, vegano, etc.)                           │
│                                                                                                                 │
│                                                                                                                 │
│  **Receta Final**: Canelones de "Ternera" XXL al Pedro Ximenez (4 porciones)                                    │
│                                                                                                                 │
│  **Ingredientes:**                                                                                              │
│  - 8 fideos de lasaña sin gluten                                                                                │
│  - 1 taza (240 ml) de leche de soja o de almendra sin azúcar                                                    │
│  - 1 taza (240 ml) de caldo de verduras                                                                         │
│  - 2 cucharadas (30 ml) de maicena                                                                              │
│  - 1 cucharada (15 ml) de harina de linaza + 3 cucharadas (45 ml) de agua (o sustituto de huevo comercial)      │
│  - 1 taza (240 g) de carne de res vegetal a base de soja o champiñones finamente texturizados                   │
│  - 1 cebolla pequeña, finamente picada                                                                          │
│  - 2 cucharadas (30 ml) de aceite de oliva                                                                      │
│  - 1 taza (140 g) de puerros finamente picados u cebollas verdes                                                │
│  - 1 taza (140 g) de coliflor picada o arroz de coliflor                                                        │
│  - 1 taza (240 ml) de jerez Pedro Ximenez                                                                       │
│  - 2 cucharadas (30 ml) de pasta de tomate                                                                      │
│  - 1 cucharadita (5 ml) de pimentón ahumado                                                                     │
│  - 1 cucharadita (5 ml) de comino                                                                               │
│  - Sal y pimienta al gusto                                                                                      │
│  - Aceite de oliva con infusión de ajo o polvo de cebolla (para sustituto de ajo)                               │
│                                                       


Crew execution result:
**Informe de Aseguramiento de Calidad**

**Validación de Cumplimiento (Usando Experiencia Dietética):**
- [ ] Todas las restricciones dietéticas estrictamente respetadas
- [ ] Sin alérgenos presentes
- [ ] Preferencias acomodadas
- [ ] No se han utilizado términos genéricos (sin gluten, sin lácteos, vegano, etc.)


**Receta Final**: Canelones de "Ternera" XXL al Pedro Ximenez (4 porciones)

**Ingredientes:**
- 8 fideos de lasaña sin gluten
- 1 taza (240 ml) de leche de soja o de almendra sin azúcar
- 1 taza (240 ml) de caldo de verduras
- 2 cucharadas (30 ml) de maicena
- 1 cucharada (15 ml) de harina de linaza + 3 cucharadas (45 ml) de agua (o sustituto de huevo comercial)
- 1 taza (240 g) de carne de res vegetal a base de soja o champiñones finamente texturizados
- 1 cebolla pequeña, finamente picada
- 2 cucharadas (30 ml) de aceite de oliva
- 1 taza (140 g) de puerros finamente picados u cebollas verdes
- 1 taza (140 g) de coliflor picada o arroz de coliflor
